In [1]:
import pandas as pd
import re

#file = '/mnt/d/Genotype Result 2/Capsicum_annuum/Pltd.snp.xls'
#file = '/mnt/c/Users/sausan.nafisah/Downloads/Pltd.snp.xls'

#ordering sample    
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)',text) ]

#main program
def genomic_db_data_preparation_software(file):
    #data manipulation
    df = pd.read_csv(file, sep = '\t')

    Chr = df['#Chr'].tolist()
    Loci = df['Loci'].tolist()
    Ref = df['REF'].tolist()
    Sequence_detail = df['Sequence_Detail'].tolist()

    #create index
    row, cols = df.shape
    index = []
    location = Chr[0]
    crop_list = ['Ca','To','Ke','Te','Me','Ja','Bm','Ka','Se','Kp','Cs']
    crop = ''

    while crop not in crop_list:
        crop = input('What crop are you going to input?\n \
        Choose one of the following:\n \
        Ca/To/Ke/Te/Me/Ja/Bm/Ka/Se/Kp/Cs\n')

    for i in range(row):
        item = '{0}_{1}_{2}'.format(crop, location, Loci[i])
        index.append(item)

    #check point
    #print('index [600]: ',index[600])

    #Obtaining the sample name
    column_name = df.columns.tolist()

    moved = ['#Chr','Loci', 'REF', 'Sequence_Detail']

    for item in moved:
        #print('removing {} from the dataset'.format(item))
        column_name.remove(item)
        
    #ordering sample 
    column_name.sort(key=natural_keys)
    #Check point
    #print (column_name)

    #preparing genotype dataset
    genotype = {}
    for column in column_name:
        #selection = '{}'.format(column)
        genotype1 = {'{}'.format(column):df['{}'.format(column)].tolist()}
        genotype.update(genotype1)
        
    #preparing csv file as Genomic db input file
    SNP_Information = {'rsID':index, 'Chr':Chr, 'Loci':Loci, 'Ref':Ref}
    Sequence_data = {'rsID':index, 'Sequencing_detail':Sequence_detail}
    Sample_Genotype = {'rsID':index}
    Sample_Genotype.update(genotype)
    
    #creating input files
    input_file1 = pd.DataFrame(SNP_Information)
    input_file1.to_csv('{}_SNP_Information.csv'.format(crop), index=False)
    print("Now you have {}_SNP_Information.csv file".format(crop))
    
    input_file2 = pd.DataFrame(Sequence_data)
    input_file2.to_csv('{}_Sequence_data.csv'.format(crop), index=False)
    print('Now you have {}_Sequence_data.csv file'.format(crop))
    
    input_file3 = pd.DataFrame(Sample_Genotype)
    input_file3.to_csv('{}_Sample_genotype.csv'.format(crop), index=False)
    print('Now you have {}_Sample_genotype.csv file'.format(crop))
    
if __name__ == "__main__":
    file = input('Enter file path or file name: ')
    genomic_db_data_preparation_software(file)

Enter file path or file name: /mnt/c/Users/sausan.nafisah/Downloads/Pltd.snp.xls
What crop are you going to input?
         Choose one of the following:
         Ca/To/Ke/Te/Me/Ja/Bm/Ka/Se/Kp/Cs
Ca
Now you have Ca_SNP_Information.csv file
Now you have Ca_Sequence_data.csv file
Now you have Ca_Sample_genotype.csv file
